## Cuaderno para ejecutar telegram-tracker-t-hoarder_tg

### Montamos nuestro drive en Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Instalamos telethon

In [ ]:
!python3 -m pip install --upgrade telethon
!

### Ejecutamos el script principal de telegram-tracket



```
usage: main.py
       [-h]
       --telegram-channel
       TELEGRAM_CHANNEL
       [--max-msgs MAX_MSGS]
       [--limit-download-to-channel-metadata]
       [--output OUTPUT]


Opciones:
 --telegram-channel  para descargar un canal
 --max-msgs Máximo número de mensajes descargados de un canal (de más recientes a más antiguos). Por defecto todos
 --limit-download-to-channel-metadata bajar solo los metadatos del canal (sin mensajes)
 --output directorio donde descargar los datos
```



### Extraer un canal

Será la raiz de la búsqueda por bola de nieve


In [ ]:
import os
# Establecer el directorio de trabajo
os.chdir("/content/drive/MyDrive/telegram-tracker-t-hoarder_tg-main")
'''
!!!!!!!  Poner nombre del canal raiz!!!!!!
'''
root_channel = 'xxxxxxxxxx'
'''
'''
path_channel = f'./data/{root_channel}'
print(f'Resultados en {path_channel}')
print(f'Descargando canal {root_channel}')
!python main.py --telegram-channel {root_channel} --output {path_channel}
# Convertir json en csv y excel y ordenar de mensaje más reciente a más antiguo
print(f'Construyendo el csv de mensajes de {root_channel}')
!python build-dataset.py --telegram-channel {root_channel} --data-path  {path_channel}
!head -1 {path_channel}/msgs_dataset.csv > {path_channel}/head_tmp.csv
!tail -n +2  {path_channel}/msgs_dataset.csv | sort -n -r -t "," --key=4 -o {path_channel}/tail_tmp.csv
!cat {path_channel}/head_tmp.csv {path_channel}/tail_tmp.csv | uniq > {path_channel}/msgs_dataset.csv
!rm {path_channel}/*tmp.*


### Dado un canal descargado, extraer canales relacionados por método bola de nieve

Atención! El canal ha debido ser previamente extraido, ejecutando la anterior casilla


In [ ]:
import os
import sys
# Establecer el directorio de trabajo
os.chdir("/content/drive/MyDrive/telegram-tracker-t-hoarder_tg-main")
'''
!!!!!!!  Poner nombre del canal raiz!!!!!!
'''
root_channel = 'xxxxxxxx'
'''
'''
path_channel = f'./data/{root_channel}'
path_channel_n2 = f'./data/{root_channel}_n2'
with open(f'{path_channel}/related_channels.csv', 'r') as inputfile:
  channels = inputfile.readlines()
  num_channels = len (channels)
i = 0
for channel in channels:
  i = i + 1
  channel = channel.strip('\n') # quitar salto de línea
  print(f'--------> downloading {channel} ({i} of {num_channels})')
  !python main.py --telegram-channel {channel} --max-msgs 400000 --output {path_channel}_n2
  # convertir json en csv y excel
  !python build-dataset.py --telegram-channel {channel} --data-path  {path_channel}_n2



### Dado un conjunto de canales buscar sus relaciones

In [ ]:
import os
import csv
# Establecer el directorio de trabajo
os.chdir("/content/drive/MyDrive/telegram-tracker-t-hoarder_tg-main")
'''
!!!!!!  Poner nombre del grupo de canales!!!!!!
'''
group = "xxxxxxxx"
'''
'''
path_group = f'./data/{group}'
if not os.path.exists(f'{path_group}/list_channels.csv'):
  os.makedirs(path_group, exist_ok=True)
  channels = input('Enter a comma separated list of channels:    ')
  channels = channels.split (",")
  with open(f'{path_group}/list_channels.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter="\n")
    writer.writerow(channels)
else:
  with open(f'{path_group}/list_channels.csv', 'r') as inputfile:
    channels = inputfile.readlines()

for channel in channels:
  channel = channel.strip('\n') # quitar salto de línea
  print(f'{channel}')
  !python main.py --telegram-channel {channel} --max-msgs 4000000 --output {path_group}
  # convertir json en csv y excel
  !python build-dataset.py --telegram-channel {channel} --data-path  {path_group}
